# viewing annihilation rates of linear fit to full
Tim Tyree<br>
10.15.2022

In [ ]:
from lib.my_initialization import *
from lib import *
#from lib.lib_care.measure.level_sets import comp_longest_level_set_and_smooth
%load_ext autoreload
%autoreload 2

import random,scipy
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.optimize import minimize
#load libraries for color bar
import matplotlib as mpl #for colorbar
from matplotlib.cm import ScalarMappable
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
#def plotter_3dpoint_cloud_sample(az,...):
from scipy import stats
#DONE: hook this routine up to dask
#DONT: hook this routine up to dask_cuda
# import scanpy as sc

import warnings
warnings.simplefilter("ignore", UserWarning)
%load_ext autoreload
%autoreload 2
# import cupy as cp
np.random.seed(42)
# cp.random.seed(42)

In [ ]:
#reset matplotlib settings
from mpltools import annotation
import matplotlib as mpl
sns.reset_orig()
mpl.rc_file_defaults()

In [ ]:
darkmode=True
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
#     theme=cuxfilter.themes.dark
#     #     theme=cuxfilter.themes.rapids
else:
    sns.set_style('white')
#     theme=cuxfilter.themes.light

In [ ]:
import panel as pn
# pn.extension('tabulator', sizing_mode="stretch_width")
# pn.extension('katex', 'mathjax')


In [ ]:
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import umap
import umap.plot

# define module

# init for interactive viewer

In [ ]:
#define function to determine environment
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
print(f"{environment()=}")
#define color palette
# PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ] #jammin
PALETTE = ["#ff6f69", "#b103fc", "#624c6b", "#27663e"] #darkula
print(f"color palette:")
pn.Row(*[pn.layout.HSpacer(height=50, background=x) for x in PALETTE])

# define dataframe pipeline

# DONE: basic viewer of mle fits

In [ ]:
#basic viewer for mle fits
kwargs = dict(data_fk_dir=f'{nb_dir}/data/full_results/data_fig4_vidmar_fk_tt.csv',
              data_lr_dir=f'{nb_dir}/data/full_results/data_fig4_vidmar_lr_tt.csv',
             min_num_particles=9)

In [ ]:
latest_run_num=29
run_num_lst = list(reversed(list(np.arange(14,latest_run_num+1))))
# run_num_lst=[25,17]
# run_num_lst=[25,19]
# run_num_lst=[25,24]
# run_num_lst = list(np.arange(12,26))
# run_num_lst = list(np.arange(26,14))
run_num_lst=[25,18]
# run_num_lst=[28,18]
run_num_lst=[28,29]
# run_num_lst=[25,29]
# run_num_lst=[30,29]
# run_num_lst=[32,29]
run_num_lst=[32,29]
#note 31==30...
# run_num_lst=[28,29]  #what is in Particle_Paper
# run_num_lst=[30,33]
# run_num_lst=[28,33]
# run_num_lst=[28,19]
run_num_lst=[28,33]
#heretim



use_filterby_neighbor=True
# neighbor=0
neighbor=1

use_filterby_minN=True
minN=9

use_filterby_amax=True
min_amax=1.

# neighbor=1
remove_no_diffusion=True
# Dinf=0
Dinf=0.01
num_digits_round_amax=3
# run_num=25 #L=5 cm
# run_num=18 #L=10 cm
gmin_fk_lst=[]
gmin_lr_lst=[]
log_dir_fk_lst=[]
log_dir_lr_lst=[]
dict_results_fk_lst=[]
dict_results_lr_lst=[]
# for run_num in [25,18]: #chosen exemplar pair
for run_num in run_num_lst:
    parquet_dir_defects_fk=f'{nb_dir}/data/osg_output/run_{run_num}_defects_to_fk.parquet'
    parquet_dir_defects_lr=f'{nb_dir}/data/osg_output/run_{run_num}_defects_to_lr.parquet'
    input_folder=f'{nb_dir}/data/osg_output/run_{run_num}_all'
#     parquet_dir_defects_fk=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_{run_num}_defects_to_fk.parquet'
#     parquet_dir_defects_lr=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_{run_num}_defects_to_lr.parquet'
#     input_folder=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_{run_num}_all'
    #load data
    df_defects_fk=pd.read_parquet(parquet_dir_defects_fk)
    df_defects_lr=pd.read_parquet(parquet_dir_defects_lr)
    
    #filterby neighbor
    if use_filterby_neighbor:
        df_defects_fk = df_defects_fk[df_defects_fk['neighbor']==neighbor].copy()
        df_defects_lr = df_defects_lr[df_defects_lr['neighbor']==neighbor].copy()
    
    #compute order parameter for each member a=amax-2*D
    #add columns for amax
    df_defects_fk['amax'] = np.around(df_defects_fk['varkappa'] + 2.*df_defects_fk['D'],num_digits_round_amax)
    df_defects_lr['amax'] = np.around(df_defects_lr['varkappa'] + 2.*df_defects_lr['D'],num_digits_round_amax)
    
    if remove_no_diffusion:
        df_defects_fk = df_defects_fk[df_defects_fk['D']>Dinf].copy()
        df_defects_lr = df_defects_lr[df_defects_lr['D']>Dinf].copy()
        
    if use_filterby_amax:
        df_defects_fk = df_defects_fk[df_defects_fk['amax']>min_amax].copy()
        df_defects_lr = df_defects_lr[df_defects_lr['amax']>min_amax].copy()
        
    
    #identify the lowest error simulation for each amax value
    print(f"\n**************************************")
    print(f"* Fenton-Karma Model (best settings):")
    print(f"**************************************")
    minrmse=9999.
    supamax=40.
    #supamax=1.8
    #supamax=40.
    for amax,g in df_defects_fk.groupby(by='amax'):
        rmse_min = g['rmse'].min()
        count = g['rmse'].count()
        log_likelihood = g['log_likelihood'].max()
        if (rmse_min<=minrmse)&(amax<=supamax):
            minrmse=rmse_min
            gmin_fk=g.copy()
            print_str =f"{amax=:.4f}, {rmse_min=:.4f}, {log_likelihood=:.4f}, {count=}"
    print(print_str)
    gmin_fk.sort_values(by='D',inplace=True)
    

    #find best simulation for the full model at this avalue
#     dg_fk=gmin_fk[gmin_fk['rmse'] == gmin_fk['rmse'].min()].copy()
    dg_fk=gmin_fk[gmin_fk['log_likelihood'] == gmin_fk['log_likelihood'].max()].copy()
    dg_fk['run']=run_num
    df_defects_fk['run']=run_num
    dg_fk['model']='Fenton-Karma'
    cluster_index,job_index = dg_fk[['cluster_index','job_index']].values.T
    input_fn = os.path.join(input_folder,f"job.out.{int(cluster_index)}.{int(job_index)}")
    #assert os.path.exists(input_fn)
    if os.path.exists(input_fn):
        input_fn_fk=input_fn
        df_fit_fk,dict_linear_fit_fk = routine_measure_annihilation_defect(input_fn,printing=True,**kwargs)
        amax_fk=dg_fk['amax'].median()
        rmse_fk=dg_fk['rmse'].median()
        L = dg_fk['L'].median()
        print(f"\nbest fit for Fenton-Karma: RMSE: {rmse_fk:.4f} Hz/cm^2, amax: {amax_fk:.4f} cm^2/s, L: {L} cm")
        print(f"{input_fn_fk=}")
        #record
        dict_results_fk=dict(
            model_name='Fenton-Karma',
            amax=amax_fk,
            rmse=rmse_fk,
            L=L,
            df_fit=df_fit_fk,
            dg_fit=dg_fk,
            dict_linear_fit=dict_linear_fit_fk,
            filename=input_fn_fk,
            gmin=gmin_fk)
        dict_results_fk_lst.append(dict_results_fk)
        gmin_fk_lst.append(gmin_fk)
        log_dir_fk_lst.append(input_fn)
    
    #identify the lowest error simulation for each amax value
    print(f"\n**************************************")
    print(f"* Luo-Rudy Model (best settings):")
    print(f"**************************************")
    minrmse=9999.
    supamax=40
    #supamax=10.2
    for amax,g in df_defects_lr.groupby(by='amax'):
        rmse_min = g['rmse'].min()
        count = g['rmse'].count()
        log_likelihood = g['log_likelihood'].max()
        if (rmse_min<=minrmse)&(amax<=supamax):
            minrmse=rmse_min
            gmin_lr=g.copy()
            print_str =f"{amax=:.4f}, {rmse_min=:.4f}, {log_likelihood=:.4f}, {count=}"
    print(print_str)        
    gmin_lr.sort_values(by='D',inplace=True)
    
    #find best simulation for the full model at this avalue
#     dg_lr=gmin_lr[gmin_lr['rmse'] == gmin_lr['rmse'].min()].copy()
    dg_lr=gmin_lr[gmin_lr['log_likelihood'] == gmin_lr['log_likelihood'].max()].copy()
    dg_lr['run']=run_num
    df_defects_fk['run']=run_num
    dg_lr['model']='Luo-Rudy'
    cluster_index,job_index = dg_lr[['cluster_index','job_index']].values.T
    input_fn = os.path.join(input_folder,f"job.out.{int(cluster_index)}.{int(job_index)}")
    #assert os.path.exists(input_fn)
    if os.path.exists(input_fn):
        input_fn_lr=input_fn
        df_fit_lr,dict_linear_fit_lr = routine_measure_annihilation_defect(input_fn,printing=True,**kwargs)
        amax_lr=dg_fk['amax'].median()
        rmse_lr=dg_lr['rmse'].median()
        L = dg_lr['L'].median()
        print(f"\nbest fit for Luo-Rudy: RMSE: {rmse_lr:.4f} Hz/cm^2, amax: {amax_lr:.4f} cm^2/s, L: {L} cm")
        print(f"{input_fn_lr=}")
        #record
        dict_results_lr=dict(
            model_name='Luo-Rudy',
            amax=amax_lr,
            rmse=rmse_lr,
            L=L,
            df_fit=df_fit_lr,
            dg_fit=dg_lr,
            dict_linear_fit=dict_linear_fit_lr,
            filename=input_fn_lr,
            gmin=gmin_lr)
        dict_results_lr_lst.append(dict_results_lr)
        gmin_lr_lst.append(gmin_lr)
        log_dir_lr_lst.append(input_fn)

#     print(f"\nparameter range supported in run #{run_num}:")
#     print(f"number of trials: {df_defects_lr.shape[0]}")
#     print(df_defects_lr.describe()[['r','kappa','D','varkappa','amax','L','neighbor']].T[['min','max']].T)

In [ ]:
# run_num=33 #all alinear estimators at L=10
# run_num=32 #all constrained to observed at L=5
# run_num=28  #what is in Particle_Paper
parquet_dir_defects_fk=f'{nb_dir}/data/osg_output/run_{run_num}_defects_to_fk.parquet'
parquet_dir_defects_lr=f'{nb_dir}/data/osg_output/run_{run_num}_defects_to_lr.parquet'
input_folder=f'{nb_dir}/data/osg_output/run_{run_num}_all'
#     parquet_dir_defects_fk=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_{run_num}_defects_to_fk.parquet'
#     parquet_dir_defects_lr=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_{run_num}_defects_to_lr.parquet'
#     input_folder=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_{run_num}_all'
#load data
df_defects_fk=pd.read_parquet(parquet_dir_defects_fk)
df_defects_lr=pd.read_parquet(parquet_dir_defects_lr)

#filterby neighbor
if use_filterby_neighbor:
    df_defects_fk = df_defects_fk[df_defects_fk['neighbor']==neighbor].copy()
    df_defects_lr = df_defects_lr[df_defects_lr['neighbor']==neighbor].copy()

#compute order parameter for each member a=amax-2*D
#add columns for amax
df_defects_fk['amax'] = np.around(df_defects_fk['varkappa'] + 2.*df_defects_fk['D'],num_digits_round_amax)
df_defects_lr['amax'] = np.around(df_defects_lr['varkappa'] + 2.*df_defects_lr['D'],num_digits_round_amax)

if remove_no_diffusion:
    df_defects_fk = df_defects_fk[df_defects_fk['D']>Dinf].copy()
    df_defects_lr = df_defects_lr[df_defects_lr['D']>Dinf].copy()

if use_filterby_amax:
    df_defects_fk = df_defects_fk[df_defects_fk['amax']>min_amax].copy()
    df_defects_lr = df_defects_lr[df_defects_lr['amax']>min_amax].copy()

In [ ]:
dict_wjr = recall_death_rates_vidmar_rappel(**kwargs)
fk=dict_wjr['fk']
lr=dict_wjr['lr']

use_filterby_minN=True
minN=10
# minN=8
# figsize = (6, 4)  #(5,4)
figsize = (4, 4)  #(5,4)
fontsize=16

# x1lim=[0.008, 11]
x1lim=[0.08, 11]
x2lim=[0.01, 250]


fig, ax = plt.subplots(figsize=figsize)



ax = plot_death_rates_loglog_full(fk,lr,ax=ax,fontsize=fontsize,x1lim=x1lim, x2lim=x2lim)

# Linear Particle (L=5)
df_fit_fk = dict_results_fk_lst[0]['df_fit'].copy()
dg_fit_fk = dict_results_fk_lst[0]['dg_fit'].copy()
df_fit_lr = dict_results_lr_lst[0]['df_fit'].copy()
dg_fit_lr = dict_results_lr_lst[0]['dg_fit'].copy()

if use_filterby_minN:
    df_fit_fk = df_fit_fk[df_fit_fk['N']>=minN].copy()
    df_fit_lr = df_fit_lr[df_fit_lr['N']>=minN].copy()

df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='k',label=r'Lin. (A=25 cm$^2$)')#f'Lin. Particle (L=5)')
df_fit_lr.plot(x='q',y='w',alpha=0.7,ax=ax,color='k',label='_Best Fit to LR')
# df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='k',label=r'Particle (A=25 cm$^2$)')#f'Lin. Particle (L=5)')
# df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='k',label=r'Linear Particle (A=25 cm$^2$)')#f'Lin. Particle (L=5)')
# df_fit_lr.plot(x='q',y='w',alpha=0.7,ax=ax,color='k',label='_Best Fit to LR')

# Linear Particle (L=10)
df_fit_fk = dict_results_fk_lst[1]['df_fit'].copy()
dg_fit_fk = dict_results_fk_lst[1]['dg_fit'].copy()
df_fit_lr = dict_results_lr_lst[1]['df_fit'].copy()
dg_fit_lr = dict_results_lr_lst[1]['dg_fit'].copy()

if use_filterby_minN:
    df_fit_fk = df_fit_fk[df_fit_fk['N']>=minN].copy()
    df_fit_lr = df_fit_lr[df_fit_lr['N']>=minN].copy()

   
df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label=r'Lin. (A=100 cm$^2$)')#f'Lin. Particle (L=5)')
df_fit_lr.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label='_Best Fit to LR')
# df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label=r'Linear Particle (A=100 cm$^2$)')#)f'Lin. Particle (L=10)')
# # df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label=f'Linear Particle (L=10)')#,label='_Best Fit to FK')
# # df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label=r'Lin. Particle (A=100 cm$^2$)')#)f'Lin. Particle (L=10)')
# # df_fit_fk.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label=r'Particle (A=100 cm$^2$)')#)f'Lin. Particle (L=10)')
# df_fit_lr.plot(x='q',y='w',alpha=0.7,ax=ax,color='r',label='_Best Fit to LR')

# format_plot
ax.tick_params(top=True, right=True,direction='in',which='both')
format_plot(ax=ax,xlabel=r'$n$ (1/cm$^2$)',ylabel=r'$w_-$ (Hz/cm$^2$)',fontsize=fontsize,use_loglog=True)
# format_plot(ax=ax,xlabel=r'$q$ (1/cm$^2$)',ylabel=r'$w_-$ (Hz/cm$^2$)',fontsize=fontsize,use_loglog=True)
# ax.legend(loc='upper left',fontsize=fontsize-2)
# ax.legend(loc='lower right',fontsize=fontsize-2.,frameon=False,ncol=2)
# leg = ax.legend(loc=(1.04,0.2),fontsize=fontsize-2.,frameon=False)
# leg = ax.legend(loc='upper left',fontsize=fontsize-3.,frameon=False,ncol=1)
leg = ax.legend(loc='lower right',fontsize=fontsize-3.,frameon=False,ncol=1)
for lh in leg.legendHandles: 
    lh.set_alpha(1)
# ax.get_legend().remove()

# x1lim = [8e-3, 1.1]  
# x2lim = [1e-2, 25]
# ax.set_xlim(x1lim)
# ax.set_ylim(x2lim)

#add annotation
# xytext=(0.09, 10)
# xytext=(0.01, 90)
# ax.annotate(r"best fit ($a>0$)", xy=xy, xytext=xytext,fontsize=fontsize)
# .tick_params(labelbottom=False, labeltop=True, labelleft=False, labelright=False,
#                      bottom=False, top=True, left=False, right=False)
# xytext=(0.01, 40)
# xy=(0.3, 11)
# ax.annotate(r"$a_{max}$ = const.", xy=xy, xytext=(0.01, 40),fontsize=fontsize-2)
# ax.annotate(r"$a_{max}$ fixed to observed", xy=xy, xytext=(0.01, 40),fontsize=fontsize-2)
# ax.annotate(r"$a_{max}$ fixed to observation"+f"\nfrom MSR of full", xy=xy, xytext=(0.01, 40),fontsize=fontsize-2)
# #add marker showing the slope of ~2
# annotation.slope_marker(origin=(.75, 2.0),
#                         slope=2,
#                         text_kwargs={'color': 'black','fontsize':16, 'alpha':0.7},
#                         poly_kwargs={'facecolor': (0.5, 0.5, 0.5), 'alpha':0.5},
#                         ax=ax)
print(f"{run_num_lst=}")
# print(f"{neighbor_lst=}")
plt.show()

In [ ]:
#heretim
#TODO: constrain ^that to desired a,D values by selecting them from df_defects_fk,lr
#TODO: plot all reasonably close fits from the linear particle model. 
# Q: do any appear to match exponents at high density?
#TODO: recompute likliehood using linearly extrapolated powerlaw from full
#TODO(later, bonus): consider making a dask dashboard with umap on the left with each point selecting a different linear particle model fit on the right

# identify the lowest error simulation for each amax value

In [ ]:
print(f"\n**************************************")
print(f"* Fenton-Karma Model (best settings):")
print(f"**************************************")
minrmse=9999.
supamax=40.
#supamax=1.8
#supamax=40.
gmin_fk_lst=[]
dict_results_fk_lst=[]
for amax,g in df_defects_fk.groupby(by='amax'):
    rmse_min = g['rmse'].min()
    count = g['rmse'].count()
    log_likelihood = g['log_likelihood'].max()
    #if (rmse_min<=minrmse)&(amax<=supamax):
    minrmse=rmse_min
    gmin_fk=g.copy()
    gmin_fk.sort_values(by='D',inplace=True)
    print_str =f"{amax=:.4f}, {rmse_min=:.4f}, {count=}, {log_likelihood=:.4f}, {gmin_fk['D'].drop_duplicates().values}"
    print(print_str)
    #TODO: identify xy values of the mle fit
        #find best simulation for the full model at this avalue
#     dg_fk=gmin_fk[gmin_fk['rmse'] == gmin_fk['rmse'].min()].copy()
    dg_fk=gmin_fk[gmin_fk['log_likelihood'] == gmin_fk['log_likelihood'].max()].copy()
    dg_fk['run']=run_num
    df_defects_fk['run']=run_num
    dg_fk['model']='Fenton-Karma'
    cluster_index,job_index = dg_fk[['cluster_index','job_index']].values.T
    input_fn = os.path.join(input_folder,f"job.out.{int(cluster_index)}.{int(job_index)}")
    #assert os.path.exists(input_fn)
    if os.path.exists(input_fn):
        input_fn_fk=input_fn
        df_fit_fk,dict_linear_fit_fk = routine_measure_annihilation_defect(input_fn,printing=True,**kwargs)
        amax_fk=dg_fk['amax'].median()
        rmse_fk=dg_fk['rmse'].median()
        L = dg_fk['L'].median()
        #print(f"- best fit RMSE: {rmse_fk:.4f} Hz/cm^2, amax: {amax_fk:.4f} cm^2/s, L: {L} cm")
        #print(f"{input_fn_fk=}")
        #record
        dict_results_fk=dict(
            model_name='Fenton-Karma',
            amax=amax_fk,
            rmse=rmse_fk,
            L=L,
            df_fit=df_fit_fk,
            dg_fit=dg_fk,
            dict_linear_fit=dict_linear_fit_fk,
            filename=input_fn_fk,
            gmin=gmin_fk)
        dict_results_fk_lst.append(dict_results_fk)
    
    #TODO: estimate powerlaw from xy values
    #Q: howdo not include q too small
    #A: try filter by min_N>10 like before 
    
    #TODO: print the exponent +/- 95%CI for the above powerlaw fit
    

        gmin_fk_lst.append(gmin_fk)


#identify the lowest error simulation for each amax value
print(f"\n**************************************")
print(f"* Luo-Rudy Model (best settings):")
print(f"**************************************")
minrmse=9999.
supamax=40.
#supamax=1.8
#supamax=40.
gmin_lr_lst=[]
dict_results_lr_lst=[]
for amax,g in df_defects_lr.groupby(by='amax'):
    rmse_min = g['rmse'].min()
    count = g['rmse'].count()
    log_likelihood = g['log_likelihood'].max()
    #if (rmse_min<=minrmse)&(amax<=supamax):
    minrmse=rmse_min
    gmin_lr=g.copy()
    gmin_lr.sort_values(by='D',inplace=True)
    print_str =f"{amax=:.4f}, {rmse_min=:.4f}, {count=}, {log_likelihood=:.4f}, {gmin_lr['D'].drop_duplicates().values}"
    print(print_str)
    #identify xy values of the mle fit
    #find best simulation for the full model at this avalue
    dg_lr=gmin_lr[gmin_lr['log_likelihood'] == gmin_lr['log_likelihood'].max()].copy()
    dg_lr['run']=run_num
    df_defects_lr['run']=run_num
    dg_lr['model']='Luo-Rudy'
    cluster_index,job_index = dg_lr[['cluster_index','job_index']].values.T
    input_fn = os.path.join(input_folder,f"job.out.{int(cluster_index)}.{int(job_index)}")
    #assert os.path.exists(input_fn)
    if os.path.exists(input_fn):
        input_fn_lr=input_fn
        df_fit_lr,dict_linear_fit_lr = routine_measure_annihilation_defect(input_fn,printing=True,**kwargs)
        amax_lr=dg_lr['amax'].median()
        rmse_lr=dg_lr['rmse'].median()
        L = dg_lr['L'].median()
        #print(f"- best fit RMSE: {rmse_lr:.4f} Hz/cm^2, amax: {amax_lr:.4f} cm^2/s, L: {L} cm")
        #record
        dict_results_lr=dict(
            model_name='Luo-Rudy',
            amax=amax_lr,
            rmse=rmse_lr,
            L=L,
            df_fit=df_fit_lr,
            dg_fit=dg_lr,
            dict_linear_fit=dict_linear_fit_lr,
            filename=input_fn_lr,
            gmin=gmin_lr)
        dict_results_lr_lst.append(dict_results_lr)
    
    #TODO: estimate powerlaw from xy values
    #Q: howdo not include q too small
    #A: try filter by min_N>10 like before 
    
    #TODO: print the exponent +/- 95%CI for the above powerlaw fit
    

        gmin_lr_lst.append(gmin_lr)


#print the best fit exponents
print(f"\n**************************************")
print(f"* the best fit exponents:")
print(f"**************************************")
min_N=9
qmax=0.7

min_N=10
# qmax=1
# qmax=1.5
# min_N=9
# qmax=0.75
# min_N=9
# qmax=0.85

# qmax=1
gmin_lst = list(gmin_fk_lst)
gmin_lst.extend(gmin_lr_lst)
dict_results_lst = list(dict_results_fk_lst)
dict_results_lst.extend(dict_results_lr_lst)
for gmin,dict_results in zip(gmin_lst,dict_results_lst):
    model_name = dict_results['model_name']
    
    dict_linear_fit = dict_results['dict_linear_fit']
    df_fit = dict_results['df_fit']
    df_fit = df_fit[df_fit['N']>min_N].copy()
    x = df_fit['q'].values
    y = df_fit['w'].values
    boo = qmax>x
    dict_fit = print_fit_power_law(x[boo],y[boo],printing=False)
    #dict_fit = print_fit_power_law(x,y,printing=False)

    # print_dict(dict_fit)
    #yhat = dict_fit['M']*x**dict_fit['m']
    # rmsehat = np.sqrt(np.mean((yhat-y)**2))
    # rmsehat #confirmed to match rmse in dict_fit
    # #DONE: plot powerlaw fit for sanity check
    # # plt.plot(x,(y-yhat)/y,'C0')
    # plt.plot(x,(y-yhat),'C0')
    # plt.plot(x,0.*x,'k--')
    # plt.show()

    r = df_fit['r'].median()
    kappa = df_fit['kappa'].median()
    D = df_fit['D'].median()
    a = df_fit['varkappa'].median()
    neighbor = df_fit['neighbor'].median()
    amax= a+2*D

     #DONE: estimate powerlaw from xy values
    #Q: howdo not include q too small
    #A: try filter by min_N>10 like before 

    #TODO: print the exponent +/- 95%CI for the above powerlaw fit
    # dict_fit['M'],dict_fit['m']
    is_model = (model_name=='Fenton-Karma')&(amax<3)
    is_model|= (model_name=='Luo-Rudy')&(amax>3)
    if is_model:
        print(f"- amax: {amax:.4f}, D: {D:.4f} --> exponent: {dict_fit['m']:.4f} +/- {dict_fit['Delta_m']:.4f} ({model_name})")
print(f"exponents were computed with {min_N=} and {qmax=}.")


# generate umap representation of trials

In [ ]:
#define the data dirs
# DATA_DIR = './data'
DATA_DIR = f'{nb_dir}/data/osg_output'
run_num=33
input_folder=f'{DATA_DIR}/run_{run_num}_all'
parquet_dir_defects_fk=f'{DATA_DIR}/run_{run_num}_defects_to_fk.parquet'
parquet_dir_defects_lr=f'{DATA_DIR}/run_{run_num}_defects_to_lr.parquet'
#load data
df_defects_fk=pd.read_parquet(parquet_dir_defects_fk)
df_defects_lr=pd.read_parquet(parquet_dir_defects_lr)
#heretim
#basic viewer for mle fits
kwargs_files = dict(
    data_fk_dir=f'{nb_dir}/data/full_results/data_fig4_vidmar_fk_tt.csv',
    data_lr_dir=f'{nb_dir}/data/full_results/data_fig4_vidmar_lr_tt.csv',
    parquet_dir_defects_fk=parquet_dir_defects_fk,
    parquet_dir_defects_lr=parquet_dir_defects_lr,
#     input_folder=input_folder,
#     run_num=run_num
#              min_num_particles=9
)

In [ ]:
xcol_lst=['amax','D','rxn','delta','neighbor']
# xcol_lst=['varkappa','D','rxn','delta','neighbor']
#extract features for training umap
df_defects_fk['rxn']=df_defects_fk['r']**2*df_defects_fk['kappa'] #cm^2/s #reaction coefficient
df_defects_fk['delta']=np.pi*df_defects_fk['r']**2*(df_defects_fk['L']**2) #cm^2/s #fraction of domain covered by one particle
df_defects_fk['amax']=df_defects_fk['varkappa']+2*df_defects_fk['D']
Xfk = df_defects_fk[xcol_lst].values
yfk = df_defects_fk['log_likelihood'].values

df_defects_lr['rxn']=df_defects_lr['r']**2*df_defects_lr['kappa'] #cm^2/s #reaction coefficient
df_defects_lr['delta']=np.pi*df_defects_lr['r']**2*(df_defects_lr['L']**2) #cm^2/s #fraction of domain covered by one particle
df_defects_lr['amax']=df_defects_lr['varkappa']+2*df_defects_lr['D']
Xlr = df_defects_lr[xcol_lst].values
ylr = df_defects_lr['log_likelihood'].values

In [ ]:
kwargs_umap = dict(
    n_neighbors=25,#15,
    n_components=2,
    metric='euclidean',
    metric_kwds=None,
    output_metric='euclidean',
    output_metric_kwds=None,
    n_epochs=None,
    learning_rate=1.0,
    init='spectral',
    min_dist=0.1,
    spread=1.0,
    random_state=42
)

In [ ]:
print(f"performing UMAP (estimated run time)...")
#train and visualize umap
model_name='Luo-Rudy'
print(f"training {model_name}...")
mapper_lr = umap.UMAP(**kwargs_umap).fit(Xlr)
# mapper_lr_supervised = umap.UMAP(**kwargs_umap).fit(Xlr,ylr)

model_name='Fenton-Karma'
print(f"training {model_name}...")
mapper_fk = umap.UMAP(**kwargs_umap).fit(Xfk)
# mapper_fk_supervised = umap.UMAP(**kwargs_umap).fit(Xfk,yfk)
# mapper_fk['model']=model_name

In [ ]:
# umap.plot.points(mapper_fk, labels=yfk
# umap.plot.points(mapper_fk, values=yfk, theme='fire')
# umap.plot.points(mapper_lr, values=Xlr[:,0], theme='fire') #amax
# umap.plot.points(mapper_lr, values=Xlr[:,1], theme='fire') #D
# umap.plot.points(mapper_lr, values=Xlr[:,2], theme='fire') #rxn
# umap.plot.points(mapper_lr, values=Xlr[:,3], theme='fire') #delta
# umap.plot.points(mapper_lr, values=Xlr[:,4]+1*126, theme='fire') #neighbor... may be all black.
# umap.plot.points(mapper_lr, values=ylr, theme='fire')
umap.plot.points(mapper_fk, values=yfk, theme='fire')
plt.show()

In [ ]:
# #TODO(later): make UMAP happen on the gpu
# from cuml.manifold import UMAP

In [ ]:
#label all of the rows with the UMAP coordinates
Xfk_umap = mapper_fk.transform(Xfk)
Xlr_umap = mapper_lr.transform(Xlr)
df_defects_fk['UMAP1']=Xfk_umap[:,0]
df_defects_fk['UMAP2']=Xfk_umap[:,1]
df_defects_lr['UMAP1']=Xlr_umap[:,0]
df_defects_lr['UMAP2']=Xlr_umap[:,1]

In [ ]:
#derive fields from log_likelihood
df_defects_fk['likelihood'] = np.exp(df_defects_fk['log_likelihood'])
df_defects_fk['rel_size'] = df_defects_fk['likelihood']/(1+df_defects_fk['likelihood']) #for markersize relative to from 0 to 1
df_defects_lr['likelihood'] = np.exp(df_defects_lr['log_likelihood'])
df_defects_lr['rel_size'] = df_defects_lr['likelihood']/(1+df_defects_lr['likelihood']) #for markersize relative to from 0 to 1

#index by run,cluster,job
dffk=df_defects_fk.copy()
dflr=df_defects_lr.copy()
dffk.set_index(['cluster_index','job_index'],inplace=True)
dflr.set_index(['cluster_index','job_index'],inplace=True)

#likelihoods from _fk and _lr into one dataframe, df_defects as df
df = dffk.copy()
df.rename({'rel_size':'rel_size_fk','likelihood':'likelihood_fk','log_likelihood':'log_likelihood_fk'},inplace=True)
df['rel_size_lr']=dflr['rel_size']
df['likelihood_lr']=dflr['likelihood']
df['log_likelihood_lr']=dflr['log_likelihood']
df.reset_index(inplace=True)
df.head(1)

In [ ]:
viewer_dir = os.path.join(input_folder,'_viewer.csv')
print(f"{viewer_dir=}")
df.to_csv(viewer_dir,index=False)

In [ ]:
df = pd.read_csv('/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_33_all/_viewer.csv')

In [ ]:
# #likelihoods from _fk and _lr into one dataframe, df_defects as df
# df = dffk.copy()
# df.rename({'rel_size':'rel_size_fk','likelihood':'likelihood_fk','log_likelihood':'log_likelihood_fk'},inplace=True)
# df['rel_size_lr']=dflr['rel_size']
# df['likelihood_lr']=dflr['likelihood']
# df['log_likelihood_lr']=dflr['log_likelihood']
# df.head()

# define panels for interactive viewing

In [ ]:
#TODO: follow walkthrough from "Make DataFrame Pipeline Interactive"
# df_defects_fk.shape,df_defects_lr.shape
if environment()=="server":
    theme="fast"
else:
    theme="simple"

In [ ]:
#TODO: dev a finished viewer
#TODO(later): abstract a finished producct for viewer to dynamically link panel left scatter via indexcol_lst
#define text fields
# textcol_lst=['r','kappa','L','varkappa','neighbor','D']
indexcol_lst=['cluster_index','job_index']
xcol_lst=['varkappa','D','r','kappa','L','neighbor']
ycol_lst=['log_likelihood_fk','log_likelihood_lr']
textcol_lst = list(xcol_lst)
textcol_lst.extend(xcol_lst)
textcol_lst.extend(ycol_lst)

todo(later): define [Panel widgets](https://panel.holoviz.org/reference/index.html#widgets)


In [ ]:
#TODO: make click get columns in xcol_lst for the selected scatter plot point
#TODO: set ^these values to panel, right using .interactive
xcol_lst

In [ ]:
#src for tutorial: https://github.com/sophiamyang/hvplot_interactive
# #TODO(later): define panel widgets
# cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)
# mfr = pn.widgets.ToggleGroup(
#     name='MFR',
#     options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'], 
#     value=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
#     button_type='success')
# yaxis = pn.widgets.RadioButtonGroup(
#     name='Y axis', 
#     options=['hp', 'weight'],
#     button_type='success'
# )
# #TODO(later): test panel widgets
# ipipeline = (
#     idf[
#         (idf.cyl == cylinders) & 
#         (idf.mfr.isin(mfr))
#     ]
#     .groupby(['origin', 'mpg'])[yaxis].mean()
#     .to_frame()
#     .reset_index()
#     .sort_values(by='mpg')  
#     .reset_index(drop=True)
# )
# ipipeline.head()

In [ ]:
# # view interactive dashboard by piping to hvplot
# ihvplot = ipipeline.hvplot(x='mpg', y=yaxis, by='origin', color=PALETTE, line_width=6, height=400)
# ihvplot

In [ ]:
# neighbors_button = pn.widgets.RadioButtonGroup(
#     name='neighbors_button', 
#     options=['vector-summed', 'neighbors'],
#     button_type='success'
# )
# cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)


# TODO: dev loader for annihilation rate results

In [ ]:
kwargs = dict(data_fk_dir=f'{nb_dir}/data/full_results/data_fig4_vidmar_fk_tt.csv',
              data_lr_dir=f'{nb_dir}/data/full_results/data_fig4_vidmar_lr_tt.csv',
             min_num_particles=10)

In [ ]:
#select the trial for panel right
run_num=33; cluster_index=24419143; job_index=36139 #noice example
cluster_index = int(cluster_index); job_index = int(job_index); 
print(f"{run_num=}, {cluster_index=}, {job_index=}")

In [ ]:
def load_annihilation_rate_results(cluster_index,job_index,input_folder,min_num_particles=10,printing=False,
                                   xcol='q',ycol='w',color='k',
                                   **kwargs):
    """
    Example Usage
p2 = load_annihilation_rate_results(cluster_index,job_index,input_folder,min_num_particles=10,
                                   xcol='q',ycol='w',color='k')
    """
    input_fn = os.path.join(input_folder,f"job.out.{int(cluster_index)}.{int(job_index)}")
    #assert os.path.exists(input_fn)
    if os.path.exists(input_fn):
        df_fit,dict_linear_fit= routine_measure_annihilation_defect(input_fn,
                                                                    printing=printing,
                                                                    min_num_particles=min_num_particles)#,**kwargs
        return df_fit.hvplot(x=xcol,y=ycol,c=color).opts(framewise=True)
    else:
        raise f"Error: the file does not exist: {input_fn}"

In [ ]:
#TODO(later): add on/off support for loglog
# use_loglog=True
# # xlabel=r'n (1/cm$^2$)',ylabel=r'$w_-$ (Hz/cm$^2$)',fontsize=fontsize
# if use_loglog:
#     pass

In [ ]:
dict_wjr = recall_death_rates_vidmar_rappel(**kwargs)
fk=dict_wjr['fk']
lr=dict_wjr['lr']

In [ ]:
use_filterby_minN=True
minN=10
# minN=8
# figsize = (6, 4)  #(5,4)
figsize = (4, 4)  #(5,4)
fontsize=16
# x1lim=[0.008, 11]
xlim=[0.08, 11]
ylim=[0.01, 250]

In [ ]:
legend_alpha=1.
minx=0.1
alpha=0.2
s=40
fontsize=18

frameon_legend=False
show_fk=True
show_lr=True
zorder=0

In [ ]:
# del p1

In [2]:
#TODO: try to get job_index from hover or click event out of sctter plot
#heretimheretim


In [ ]:
#define plot of full model annihilation rates as a holoview panel base 
width=400;height=700
xcol='q'
ycol='w'
color_lst_mpldefault=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
#plot the full models

if show_fk:
    boofk=fk['q']>=minx
    pfk=fk[boofk].hvplot.scatter(x=xcol,y=ycol,c=color_lst_mpldefault[0],
                           alpha=alpha,s=s,label='Fenton-Karma',
                                  hover=False,
                                 legend='bottom',rot=45,
                                    width=width,height=height,
                                    logx=True,logy=True,
#                                  xlabel=r'$n$ (1/cm$^2$)',ylabel=r'$w_-$ (Hz/cm$^2$)',
                                    xlabel=r'n (1/cm^2)',ylabel=r'w (Hz/cm^2)',
                                    xlim=xlim,ylim=ylim)
#                           )#.opts(framewise=True)
if show_lr:
    boolr=lr['q']>=minx
    plr=lr[boolr].hvplot.scatter(x=xcol,y=ycol,c=color_lst_mpldefault[1],
                                    alpha=alpha,s=s,label='Luo-Rudy',
                                    hover=False)
#                                  xlabel=r'$n$ (1/cm$^2$)',ylabel=r'$w_-$ (Hz/cm$^2$)'
#                                  xlabel = #xlabel/ylabel/clabel (default=None): str
#                                  xticks/yticks
#                                  responsive=True, #whether the plot should responsively resize depending on the  size of the browser
#                           )#.opts(framewise=True)
    
p2_base = pfk*plr
# p2_base = plr*pfk

In [ ]:
min_num_particles=10 #8
printing=True
#TODO: add title with the selected linear particle model parameters
title=f"{textcol_lst}"
print(title)
# # title=f"{textcol_lst_values}"
# p2_selected = load_annihilation_rate_results(cluster_index,job_index,input_folder,min_num_particles=min_num_particles,printing=printing,
#                                    xcol='q',ycol='w',color='k',title=title)

# # p2 = df_fit.hvplot(x='q',y='w',c='k').opts(framewise=True)
# # type(p2)
# p2 = p2_base*p2_selected
# p2 #uncomment to preview

In [ ]:
def loader_annihilation_rate_results(cluster_index,job_index):
    return load_annihilation_rate_results(cluster_index,job_index,input_folder,min_num_particles=min_num_particles,printing=printing,
                                   xcol='q',ycol='w',color='k',title=title)

In [ ]:
max_job_index = df['job_index'].max()
cluster_index_values = df['cluster_index'].drop_duplicates().values.flatten()
job_index_values = df['job_index'].drop_duplicates().values.flatten()
cluster_index_values,max_job_index

In [ ]:
job_index,cluster_index
max_job_index

In [ ]:
# # define dataframe for interactive viewing
# idf = df.interactive()


In [ ]:
# symbol = pn.widgets.RadioButtonGroup(options=stock_symbols)
cluster_dropdown = pn.widgets.Select(name="Cluster:", options=list(cluster_index_values))# value=24419143)
# job_dropdown = pn.widgets.Select(name="Job:", options=list(cluster_index_values))
# job_dropdown = pn.widgets.Select(name="Cluster:", options=list(job_index_values))# value=24419143)
job_input = pn.widgets.IntInput(name='Job:',value=job_index)
# job_index_slider = pn.widgets.IntSlider(name="Job:", value=36139, start=1, end=max_job_index)

p_widgets = pn.Column(cluster_dropdown,job_input)

In [ ]:
#DONE: make p1 show index upon hover
#DONE: make p1 show list of data upon hover
#TODO: make p1 dynamically update a the selected index
ccol='log_likelihood_lr'
max_markersize=1.
scol='size'
df[scol]=(df['rel_size_lr']+1)*max_markersize
p1 = df.hvplot.scatter(
    x="UMAP1",
    y="UMAP2",
    s=scol,
    cmap='fire',
    c=ccol,#clim=[vmin,vmax],
    colorbar=False,
    fontscale=1., #default
#     grid=True,
    height=height,
    width=height,
    hover=True,
    hover_cols=textcol_lst,
)#, height=400, width=400)
#project=True)
# project (default=False):
#     Whether to project the data before plotting (adds initial
#     overhead but avoids projecting data when plot is dynamically
#     updated).
# p1

In [ ]:
# # p_widgets
# p1

In [ ]:
# rolling_window = pn.widgets.IntSlider(name='Rolling Window', value=10, start=1, end=365)

# print(f"the widget will look like the following:")
# pn.Column(symbol, variable, rolling_window)
# dmap = hv.DynamicMap(loader_annihilation_rate_results,
#                     cluster_index=cluster_dropdown, 
#                     job_index=job_input)#,input_folder=input_folder)
# dmap
# dmap = hv.DynamicMap(load_annihilation_rate_results,
#                      kdims=['cluster_index','job_index',input_folder])
#DONE: replace [job_index] with all jobs available: 
#DONE: replace [cluster_index] with all jobs available: 
# dmap = dmap.redim.values(cluster_index=[cluster_index], job_index=[job_index])



In [ ]:
# p2_selected = dmap ## does this work??
#Q: does ^this loop like p2_selected?
#if A==Yes, smack it in as  the pout = p1 + p2*p2_selected
dmap = hv.DynamicMap(loader_annihilation_rate_results,
                     kdims=['cluster_index','job_index'])
dmap = dmap.redim.values(cluster_index=list(cluster_index_values), job_index=list(job_index_values))
dmap.opts(framewise=True)
p2_selected = dmap


In [ ]:
# # pn.template.DarkTheme.
# pn.template.FastListTemplate(
#     site="Panel", title="Folium", main=["This app demonstrates **how to use Folium with Panel**.", 
#                                         PanelFoliumMap().view])#.servable()


In [ ]:
# run_num

In [ ]:
p2 = p2_base*p2_selected
# pout = p1+p2*p2_selected
# pout
app =  pn.Row(p1,p2*p2_selected)

In [ ]:
# #TODO: use tile to merge ^these two panels instead of +
# #TODO: cursory search for linking panels using panel.tile? as tile.
app

In [ ]:
#this .html file takes forever to write, and it is huge...
# app.save(f"dashboard_{run_num}.html", embed=True)

In [ ]:
# dmap = hv.DynamicMap(pn.bind(load_symbol, symbol=symbol, variable=variable))
# smoothed = rolling(dmap, rolling_window=rolling_window)

# app = pn.Row(pn.WidgetBox('## Stock Explorer', symbol, variable, rolling_window), 
#              smoothed.opts(width=500, framewise=True)).servable()
# app

In [ ]:
# pn.template.react.ReactDarkTheme


In [ ]:
# load_annihilation_rate_results
#TODO: dynamically set annihilation trial on right panel
# stock_symbols = ['AAPL', 'IBM', 'FB', 'GOOG', 'MSFT']
# variables = ['open', 'high', 'low', 'close', 'volume', 'adj_close']

app = pn.Row(pn.WidgetBox('## Linear Particle Model Explorer', cluster_dropdown, job_input), 
             dmap.opts(width=500, framewise=True)).servable()
app
#TODO: use ^these to update p2 dynamically

#GOAL: figure out scatter plot point selection
#TODO: search for a way to use hvplot.scatter to select a value from a scatter plot dynamically
#TODO: search for a way to use cuxfilter/datashader to select a value from a scatter plot dynamically
#heretimheretim
#Q: wasn't it something ... .bind( ...?

In [ ]:
# max_job_index=10
# pn.widgets.IntSlider?

In [ ]:
# pn.widgets.Select?

In [ ]:
# app = pn.Row(pn.WidgetBox('## Stock Explorer', symbol, variable, rolling_window), 
#              smoothed.opts(width=500, framewise=True)).servable()
# app

In [ ]:
#TODO(later): make legend markers easier to see
#HINT:
# leg = ax.legend(loc='lower right',fontsize=fontsize-3.,frameon=False,ncol=1)
# for lh in leg.legendHandles: 
#     lh.set_alpha(1)

In [ ]:
textcol_lst=[#'cluster_index','job_index'
             'amax','D','r','kappa','L','neighbor',
#              'log_likelihood_fk','log_likelihood_lr'
            ]

In [ ]:
#TODO(later): change backend of p1 plotting to cudf... it should be as easy as cudf.DataFrame?
# import cudf
# >>> unknown cuda error
#TODO(later): fix cudf not importing in this ipython environment...
# gf=cudf.DataFrame(df)

In [ ]:
# #DONE: format panel p2
# p2.options
# p2.param
# p2.params
# p2.values
# p2.set_param
# p2.update
# p2.relabel?
# p2.redim.range?

In [ ]:
# hv.help(the_holoviews_object)

# TODO: dev declaritive viewer from src:


In [ ]:
#TODO: add selection functionality from panel left to panel right
p1+p2*p2_selected

In [ ]:
# import panel as pn

symbol = pn.widgets.RadioButtonGroup(options=stock_symbols)
variable = pn.widgets.Select(options=variables)
rolling_window = pn.widgets.IntSlider(name='Rolling Window', value=10, start=1, end=365)

pn.Column(symbol, variable, rolling_window)

In [ ]:
# # dmap = hv.DynamicMap(pn.bind(load_symbol, symbol=symbol, variable=variable))
# # smoothed = rolling(dmap, rolling_window=rolling_window)

# app = pn.Row(pn.WidgetBox('## Stock Explorer', symbol, variable, rolling_window), 
#              smoothed.opts(width=500, framewise=True)).servable()
# app

In [ ]:

## Make DataFrame Pipeline Interactive
#TODO: dev figure rightin hvplot
# xcol_lst

In [ ]:
# #to serve ^this dashboard through HINT, uncomment and then run something like hin
# template = pn.template.FastListTemplate(
#     title='Interactive DataFrame Dashboards with hvplot .interactive', 
#     sidebar=[cylinders, 'Manufacturers', mfr, 'Y axis' , yaxis],
#     main=[ihvplot.panel(), itable.panel()],
#     accent_base_color="#88d8b0",
#     header_background="#88d8b0",
# )
# # template.show()
# template.servable();

HINT: To get the Tabulator table styled nicely for dark mode you can set `theme='fast'` when you define the `itable`. It won't work in the notebook though.

To *serve the notebook* run `panel serve hvplot_interactive.ipynb`.

In [ ]:
#heretim
#DONE: looked for any dtypes that need fixin
# ['num_obs', 'qmin', 'qmax', 'rmse', 'log_likelihood', 'emin', 'emax',
#        'lemin', 'lemax', 'no_repulsion', 'dt', 'L', 'r', 'set_second', 'kappa',
#        'D', 'reflect', 'neighbor', 'x0', 'varkappa', 'no_attraction', 'Dt',
#        'niter', 'force_code', 'q_lo', 'q_hi', 'N_lo', 'N_hi', 'w_lo', 'w_hi',
#        'CollTime_lo', 'CollTime_hi', 'rxn', 'delta', 'amax', 'UMAP1', 'UMAP2',
#        'likelihood', 'rel_size', 'rel_size_lr', 'likelihood_lr',
#        'log_likelihood_lr']
# col='likelihood_lr'
# col='cluster_index'
# df[col].dtype
#note: cluster_index,job_index columns do not exist because i elevated them together as the index
# query = df['neighbor']==neighbor


In [ ]:
# 'reflect', 'neighbor',

In [ ]:
#TODO: make textcol_lst visible upon hover
textcol_lst

#TODO: make click get columns in xcol_lst for the selected scatter plot point
#TODO: set ^these values to panel, right using .interactive
xcol_lst

In [ ]:
#TODO: dev map from row in X=Xlr to plot of (black,red) the annihilation rates over (blue,orange) those of the full models
i=137
X=Xlr
df_defects = df_defects_lr
# xcol_lst = ['amax', 'D', 'rxn', 'delta', 'neighbor']
# def foo
xvals= X[i]
# amax,D,rxn,delta,neighbor = vals
query = True
for x,xcol in zip(xvals,xcol_lst):
    query &= df_defects[xcol]==x
num_selected=np.sum(query)
print(f"{num_selected=}")

In [ ]:
testing=True
if testing:
    #test that precisely one job is selected
    assert num_selected==1 
    
#select the job
cluster_index,job_index = df_defects[query][['cluster_index','job_index']].values.T#.tolist()
cluster_index = int(cluster_index); job_index = int(job_index); 
print(f"{run_num=}, {cluster_index=}, {job_index=}")

#TODO: recall the directory from the base_folder belonging to model_name
#inputs: base_folder belonging to osg grid searches
#inputs: run_num,cluster_index,job_index

In [ ]:
p1 = df_defects_fk.hvplot.scatter(x='UMAP1', y='UMAP2',label='Fenton-Karma')#, **kwargs)
p2 = df_defects_lr.hvplot.scatter(x='UMAP1', y='UMAP2',label='Luo-Rudy',
                                  cmap='jet',c='amax',s='size'
                                 )#, **kwargs)
print(p1+p2)
#heretim
#TODO: dev hvplot of what I want on panel left
#TODO: choose a trial point from it using .interactive as shown below
# p1+p2
p2
p1

#TODO: hv plot this run's annihilation rates
#TODO: hv plot the full models' annihilation rates
#TODO: compose these
#TODO: view these and verify looks reasonable
#TODO: compose these two into a dashboard
#TODO: modify ^that dashboard to use .interactive() to update panel right from  default run_num=33, cluster_index=24419143, job_index=21754 
print(f"{run_num=}, {cluster_index=}, {job_index=}")

In [ ]:
df_defects.columns

# TODO: dev hvplot minimalist dashboard

In [ ]:
# #DONT: use two cudf dataframes in one dashboard...
# # #move dataframes to gpu
# dffk=cudf.DataFrame(df_defects_fk)
# dflr=cudf.DataFrame(df_defects_lr)
# # dffk.shape,dflr.shape

# # cux_df = cuxfilter.DataFrame.from_arrow(DATA_DIR+'/auto_accidents.arrow')


In [ ]:
# from bokeh.sampledata.autompg import autompg_clean as df
import hvplot.pandas
import hvplot.pandas
import panel as pn
import holoviews as hv
hv.extension('bokeh')
pn.extension('tabulator')
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]

In [ ]:
hv.extension('bokeh')
pn.extension('tabulator')
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
# view rawhvplot_interactive2.py hosted with ❤ by GitHub

In [ ]:
max_markersize=50.
df_defects_fk['likelihood'] = np.exp(df_defects_fk['log_likelihood'])
df_defects_fk['size'] = max_markersize*df_defects_fk['likelihood']/(1+df_defects_fk['likelihood'])
df_defects_lr['likelihood'] = np.exp(df_defects_lr['log_likelihood'])
df_defects_lr['size'] = max_markersize*df_defects_lr['likelihood']/(1+df_defects_lr['likelihood'])
#heretim

In [ ]:

p1 = df_defects_fk.hvplot.scatter(x='UMAP1', y='UMAP2',label='Fenton-Karma')#, **kwargs)
p2 = df_defects_lr.hvplot.scatter(x='UMAP1', y='UMAP2',label='Luo-Rudy',
                                  cmap='jet',c='amax',s='size'
                                 )#, **kwargs)
print(p1+p2)
#heretim
#TODO: dev hvplot of what I want on panel left
#TODO: choose a trial point from it using .interactive as shown below
# p1+p2
p2
p1

In [ ]:
#TODO: hv plot this run's annihilation rates
#TODO: hv plot the full models' annihilation rates
#TODO: compose these
#TODO: view these and verify looks reasonable
#TODO: compose these two into a dashboard
#TODO: modify ^that dashboard to use .interactive() to update panel right from  default run_num=33, cluster_index=24419143, job_index=21754 
print(f"{run_num=}, {cluster_index=}, {job_index=}")


In [ ]:

# hv.show()
# p1.pprint()

In [ ]:
#TODO: dev hvplot of what I want on panel right

In [ ]:
# #TODO: dev map from my data to data i want to see plotted:
# #HINT: 

# # Make DataFrame Pipeline Interactive
# idf = df.interactive()

# # Define Panel widgets
# cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)
# mfr = pn.widgets.ToggleGroup(
#     name='MFR',
#     options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'], 
#     value=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
#     button_type='success')
# yaxis = pn.widgets.RadioButtonGroup(
#     name='Y axis', 
#     options=['hp', 'weight'],
#     button_type='success'
# )

# # Combine pipeline and widgets
# ipipeline = (
#     idf[
#         (idf.cyl == cylinders) & 
#         (idf.mfr.isin(mfr))
#     ]
#     .groupby(['origin', 'mpg'])[yaxis].mean()
#     .to_frame()
#     .reset_index()
#     .sort_values(by='mpg')  
#     .reset_index(drop=True)
# )

In [ ]:
# #HINT: build dashboard
# #Layout using Template
# template = pn.template.FastListTemplate(
#     title='Interactive DataFrame Dashboards with hvplot .interactive', 
#     sidebar=[cylinders, 'Manufacturers', mfr, 'Y axis' , yaxis],
#     main=[ihvplot.panel(), itable.panel()],
#     accent_base_color="#88d8b0",
#     header_background="#88d8b0",
# )
# # template.show()
# template.servable();

In [ ]:
print(f"HINT: deploy a dashboard, simple run in terminal:")
print(f"$ panel serve hvplot_interactive.ipynb")

In [ ]:
# holoviews.Scatter?
# chart_hs = holoviews.Scatter(data=dflr, kdims=['UMAP1'], vdims=['UMAP2'],label='LR')#, **kwargs)
# type(chart_hs)
# chart_hs

# scratchwerk

In [ ]:
import cudf
from bokeh.server.server import Server
import cuxfilter
import sys

In [ ]:
#NOTE: this will all be way easier if i do the full models separately.
# df = cudf.read_parquet('./data/census_data.parquet/*')
# df=cudf.DataFrame(df_defects_fk)
df=cudf.DataFrame(df_defects_lr)
df.reset_index(inplace=True)
df['indexcol']=df.index.values
# df=cudf.DataFrame(df_defects)
#create cuxfilter dataframe
cux_df = cuxfilter.DataFrame.from_dataframe(df)
# df_defects_lr['UMAP1']

In [ ]:
chart0 = cuxfilter.charts.view_dataframe(textcol_lst)#['UMAP1', 'UMAP2'])



#declare dashboard
# d = cux_df.dashboard([chart0], title='UMAP Dashboard')

# cuxfilter.load_notebook_assets()
# chart0.view()

In [ ]:
import random
# dffk['UMAP1']

In [ ]:
# dffk[['UMAP1','UMAP2']].to_pandas().describe()

In [ ]:
# # dffk[['UMAP1','UMAP2']].describe()
# #src: https://docs.rapids.ai/api/cuxfilter/stable/charts/datashader_charts.html#scatter-chart
# #heretim
# #TODO: get this example working
# #TODO: modify this example to show the annihilation rates overlaid on the full model...
# from cuxfilter import DataFrame
# from cuxfilter.charts.datashader import line
# import numpy as np
# import cudf
# import random
# import cuxfilter

# n = 100000                           # Number of points
# start = 1456297053                   # Start time
# end = start + 60 * 60 * 24

# cux_df_ = DataFrame.from_dataframe(cudf.DataFrame({'x': np.linspace(start, end, n), 'y':np.random.normal(0, 0.3, size=n).cumsum() + 50}))
# line_chart_1 = line(x='x', y='y', unselected_alpha=0.2)

# d = cux_df_.dashboard([line_chart_1])
# line_chart_1.view()

In [ ]:
# cuxfilter.assets.geojson_mapper.geo_json_mapper

In [ ]:
# cuxfilter.datatile.BaseChart?
# cuxfilter.charts.bokeh.bokeh.plots.Line


In [ ]:
# cux_df.dashboard?
# cuxfilter.charts.scatter?
# cuxfilter.charts.scatter?

In [ ]:
scatter_chart = cuxfilter.charts.scatter(x='UMAP1',y='UMAP2',aggregate_col='amax')#, pixel_shade_type="linear", unselected_alpha=0.2)
# scatter_chart = cuxfilter.charts.scatter(x='UMAP1',y='UMAP2',aggregate_col='indexcol')#, pixel_shade_type="linear", unselected_alpha=0.2)
d = cux_df.dashboard([scatter_chart])
scatter_chart.view()

In [ ]:
from cuxfilter import DataFrame
from cuxfilter.charts import scatter
import cudf
import random

# cux_df = DataFrame.from_dataframe(cudf.DataFrame({'x': [float(random.randrange(-8239000,-8229000)) for i in range(10000)], 'y':[float(random.randrange(4960000, 4980000)) for i in range(10000)]}))
# setting pixel_shade_type='linear' to display legend (currently supports only log/linear)
# scatter_chart = scatter(x='x',y='y')#, pixel_shade_type="linear", unselected_alpha=0.2)

# d = cux_df.dashboard([scatter_chart])
d = cux_df.dashboard(
    [scatter_chart],
    sidebar: ['UMAP1'],
    layout=<class 'cuxfilter.layouts.layouts.Layout0'>,
    theme=<class 'cuxfilter.themes.light.LightTheme'>,
    title='Dashboard',
    data_size_widget=True,
    warnings=False,
    layout_array=None,
)
scatter_chart.view()


In [ ]:
#CONFIRMED: scatter plot is failing to function because of something completely independent of me.
#src: https://docs.rapids.ai/api/cuxfilter/legacy/charts/datashader_charts.html
from cuxfilter import DataFrame
from cuxfilter.charts import scatter
import cudf
import random

cux_df = DataFrame.from_dataframe(cudf.DataFrame({'x': [float(random.randrange(-8239000,-8229000)) for i in range(10000)], 'y':[float(random.randrange(4960000, 4980000)) for i in range(10000)]}))
# setting pixel_shade_type='linear' to display legend (currently supports only log/linear)
scatter_chart = scatter(x='x',y='y', pixel_shade_type="linear", unselected_alpha=0.2)

# fig = px.scatter(df, x="gdp per capita", y="life expectancy",
#                  size="population", color="continent", hover_name="country",
#                  log_x=True, size_max=60)

d = cux_df.dashboard([scatter_chart])
scatter_chart.view()

In [ ]:
# cuxfilter.charts.scatter/?
# from cuxfilter import DataFrame
# from cuxfilter.charts import scatter
# import cudf
# cux_df = cuxfilter.DataFrame.from_dataframe(dffk)
cux_df = cuxfilter.DataFrame.from_dataframe(dflr)
# cux_df = DataFrame.from_dataframe(cudf.DataFrame({'x': [float(random.randrange(-8239000,-8229000)) for i in range(10000)], 'y':[float(random.randrange(4960000, 4980000)) for i in range(10000)]}))
# chart1=cuxfilter.charts.datashader.scatter('UMAP1','UMAP2')


# setting pixel_shade_type='linear' to display legend (currently supports only log/linear)
# scatter_chart = cuxfilter.charts.scatter(x='amax',y='D')#, pixel_shade_type="linear", unselected_alpha=0.2)
# scatter_chart = cuxfilter.charts.scatter(x='UMAP1',y='UMAP2', pixel_shade_type="linear", unselected_alpha=0.2)

# d = cux_df.dashboard([scatter_chart])
# scatter_chart.view()

In [ ]:
import holoviews

In [ ]:
# # Run this app with `python app.py` and
# # visit http://127.0.0.1:8050/ in your web browser.

# from dash import Dash, dcc, html
# import plotly.express as px
# import pandas as pd


# app = Dash(__name__)

# df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

# fig = px.scatter(df, x="gdp per capita", y="life expectancy",
#                  size="population", color="continent", hover_name="country",
#                  log_x=True, size_max=60)

In [ ]:
# holoviews.Scatter?
chart_hs = holoviews.Scatter(data=dflr, kdims=['UMAP1'], vdims=['UMAP2'],label='LR')#, **kwargs)
type(chart_hs)
# chart_hs

In [ ]:
await d.preview()

In [ ]:
#TODO: add a line chart
# cux_df = cuxfilter.DataFrame.from_dataframe(cudf.DataFrame({'key': [0, 1, 2, 3, 4], 'val':[float(i + 10) for i in range(5)]}))
# line_chart_1 = cuxfilter.charts.line('key', 'val', data_points=5, add_interaction=False)
line_chart_1 = cuxfilter.charts.line(x='UMAP1', y='UMAP2', data_points=5, add_interaction=False)

d = cux_df.dashboard([line_chart_1])
#view the individual bar chart part of the dashboard d
await d.preview()
# line_chart_1.view()

In [ ]:
# # tile_provider
# cuxfilter.charts.scatter?

In [ ]:
#TODO: add a table chart
#src: https://docs.rapids.ai/api/cuxfilter/stable/charts/custom_charts.html#table-view-chart
import numpy as np
import cudf
import cuxfilter

geoJSONSource='https://raw.githubusercontent.com/rapidsai/cuxfilter/GTC-2018-mortgage-visualization/javascript/demos/GTC%20demo/src/data/zip3-ms-rhs-lessprops.json'
size = 1000

cux_df = cuxfilter.DataFrame.from_dataframe(
    cudf.DataFrame({
                    '_color':np.random.randint(20,30, size=size*10)/100,
                    'zip': list(np.arange(1,1001))*10,
                    'elevation': np.random.randint(0,1000, size=size*10)
    })
)

chart0 = cuxfilter.charts.view_dataframe(['zip', 'elevation'])
# chart0 = cuxfilter.charts.scatter('zip', 'elevation')

#declare dashboard
d = cux_df.dashboard([chart0], title='Mortgage Dashboard')

# cuxfilter.load_notebook_assets()
chart0.view()

In [ ]:
import dash
import plotly

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# # app = Dash(__name__)
# app = Dash()
# app.run_server(debug=True)
# df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')
df = df_defects_lr.copy()
fig = px.scatter(df, x="UMAP1", y="UMAP2",
                 size="D", color="rxn", hover_name="amax",
                 log_x=False, size_max=6)



# if __name__ == '__main__':
#     app.run_server(debug=True)
fig.show()

In [ ]:
# app.

In [ ]:
import plotly.graph_objects as go # or plotly.express as px
# fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )

import dash
import dash_core_components as dcc
import dash_html_components as html


In [ ]:

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, 
               port='8053',#'8050',
               use_reloader=False)  # Turn off reloader if inside Jupyter

In [ ]:
# app.run?

In [ ]:
# app = Dash(__name__)
app = Dash()
app.layout = html.Div([
    dcc.Graph(
        id='amax',
        figure=fig
    )
])
# app.run_server(debug=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')
from holoviews import opts
from holoviews.plotting.links import DataLink
# import hvplot.pandas

df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df['Even'] = df['D']%2 == 0

vdims = [('A', 'A')]
ds = hv.Dataset(df, ['B','Even'], vdims)
#ds = ds.aggregate(function=np.mean) this does nothing here, in my code it takes the mean of 100 or so data points in each group for each year.

scat = hv.Scatter(ds,'B','A')
layout = ds.to(hv.Scatter,'B','A') * hv.Slope.from_scatter(scat)
layout.opts(opts.Curve(width=800, height=400, framewise=True))

In [ ]:
#TODO: turn this into a holoview widget that selects the trial
#src: https://holoviews.org/user_guide/Dashboards.html
dmap = hv.DynamicMap(pn.bind(load_symbol, symbol=symbol, variable=variable))
smoothed = rolling(dmap, rolling_window=rolling_window)

app = pn.Row(pn.WidgetBox('## Stock Explorer', symbol, variable, rolling_window), 
             smoothed.opts(width=500, framewise=True)).servable()
app

In [ ]:
# hv.DynamicMap?

In [ ]:
#heretim
#TODO: use cuxfilter.charts.scatter as chart1
# cuxfilter.charts.scatter?
# chart1=cuxfilter.charts.datashader.scatter('UMAP1','UMAP2')
#     x='UMAP1',
#     y='UMAP2')#
#     x_range=None,
#     y_range=None,
#     add_interaction=True,
#     color_palette=None,
#     aggregate_col=None,
#     aggregate_fn='count',
#     point_size=15,
#     point_shape='circle',
#     pixel_shade_type='eq_hist',
#     pixel_density=0.5,
#     pixel_spread='dynspread',
#     tile_provider=None,
#     width=800,
#     height=400,
#     title='',
#     timeout=100,
#     legend=True,
#     legend_position='top_right',
#     unselected_alpha=0.2,
#     **library_specific_params,
# )

In [ ]:
# chart1 = cuxfilter.charts.bar('amax')
chart2 = cuxfilter.charts.bar('rxn')

In [ ]:
# declare dashboard
# charts_list = [chart1, chart2]
charts_list = [chart1, chart2]
d = cux_df.dashboard(charts_list, layout=cuxfilter.layouts.feature_and_base, theme=cuxfilter.themes.rapids, title="Data Layout: Custom (as left feature right double)")

# d = cux_df.dashboard(charts_list, 
#                      layout=cuxfilter.layouts.feature_and_double_base,
#                      theme =cuxfilter.themes.rapids, #cuxfilter.themes.light, 
#                      title='Dataset')
# d = cux_df.dashboard(charts_list[:4], layout=cuxfilter.layouts.left_feature_right_double, theme=cuxfilter.themes.rapids, title="Layout - left feature right double")
await d.preview()

In [ ]:
# chart1 = cuxfilter.charts.scatter(x='amax', y='log_likelihood')
# chart1 = cuxfilter.charts.bar('amax')
chart2 = cuxfilter.charts.bar('rxn')
chart3 = cuxfilter.charts.bar('delta')
chart4 = cuxfilter.charts.bar('log_likelihood')
# d = cux_df.dashboard([chart0, chart1, chart2], layout=cuxfilter.layouts.feature_and_double_base)
# d.show()

#TODO: modify this example to host a multiuser server dashboard for this
# [chart0, chart1, chart2], layout=cuxfilter.layouts.feature_and_double_base

# cuxfilter.layouts.feature_and_base?
charts_list = [
    chart1, chart2,
    chart3, chart4,
]
# declare dashboard
charts_list = [chart1, chart2, chart3, chart4]
d = cux_df.dashboard(charts_list, layout_array=[
    [1, 1, 2, 2],
    [1, 1, 3, 4]
    ], theme=cuxfilter.themes.rapids, title="Data Layout: Custom (as left feature right double)")

# d = cux_df.dashboard(charts_list, 
#                      layout=cuxfilter.layouts.feature_and_double_base,
#                      theme =cuxfilter.themes.rapids, #cuxfilter.themes.light, 
#                      title='Dataset')
# d = cux_df.dashboard(charts_list[:4], layout=cuxfilter.layouts.left_feature_right_double, theme=cuxfilter.themes.rapids, title="Layout - left feature right double")
await d.preview()

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


app = Dash(__name__)

df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)

app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# # cuxfilter.charts.datashader.scatter?
# cuxfilter.charts.datashader.scatter(
#     x,
#     y,
#     x_range=None,
#     y_range=None,
#     add_interaction=True,
#     color_palette=None,
#     aggregate_col=None,
#     aggregate_fn='count',
#     point_size=15,
#     point_shape='circle',
#     pixel_shade_type='eq_hist',
#     pixel_density=0.5,
#     pixel_spread='dynspread',
#     tile_provider=None,
#     width=800,
#     height=400,
#     title='',
#     timeout=100,
#     legend=True,
#     legend_position='top_right',
#     unselected_alpha=0.2,
#     **library_specific_params,
# )

In [ ]:
#  NameError: name 'port' is not defined 
# ---------------------------------------------------------------------------

# def bkapp(doc):
#     global cux_df
#     #_________________________________________________
# chart1 = cuxfilter.charts.scatter(x='amax', y='log_likelihood')
chart1 = cuxfilter.charts.bar('amax')
chart2 = cuxfilter.charts.bar('rxn')
chart3 = cuxfilter.charts.bar('delta')
chart4 = cuxfilter.charts.bar('log_likelihood')
# d = cux_df.dashboard([chart0, chart1, chart2], layout=cuxfilter.layouts.feature_and_double_base)
# d.show()

#TODO: modify this example to host a multiuser server dashboard for this
# [chart0, chart1, chart2], layout=cuxfilter.layouts.feature_and_double_base

# cuxfilter.layouts.feature_and_base?
charts_list = [
    chart1, chart2,
    chart3, chart4,
]
# declare dashboard
charts_list = [chart1, chart2, chart3, chart4]
d = cux_df.dashboard(charts_list, layout_array=[
    [1, 1, 2, 2],
    [1, 1, 3, 4]
    ], theme=cuxfilter.themes.rapids, title="Data Layout: Custom (as left feature right double)")
#     #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#         # run the dashboard as a webapp:
#     d._dashboard.generate_dashboard(
#         d.title, d._charts, d._theme
#     ).server_doc(doc)

In [ ]:
await d.preview()

In [ ]:
d.show()

In [ ]:
#  NameError: name 'port' is not defined 
# ---------------------------------------------------------------------------
server = Server({'/custom_dashboard': bkapp}, num_procs=1, allow_websocket_origin=["127.0.0.1:80"])
server.start()

print('running server on port '+str(port))
server.io_loop.start()

In [ ]:
    
    # run the dashboard as a webapp:
    d._dashboard.generate_dashboard(
        d.title, d._charts, d._theme
    ).server_doc(doc)



In [ ]:
# #TODO: plot multiuser dashboard for these dataframes
# # cux_df = cuxfilter.DataFrame.from_arrow(DATA_DIR+'/auto_accidents.arrow')
# # cux_df = cuxfilter.DataFrame.from_parquet(parquet_dir_defects_fk)
# cux_df = cuxfilter.DataFrame.from_parquet(parquet_dir_defects_lr)
# # cux_df.data['ST_CASE'] = cux_df.data['ST_CASE'].astype('float64')

# def bkapp(doc):
#     global cux_df
#     label_map = {1: 'Sunday',    2: 'Monday',    3: 'Tuesday',    4: 'Wednesday',   5: 'Thursday',    6: 'Friday',    7: 'Saturday',    9: 'Unknown'}
#     gtc_demo_red_blue_palette = [ (49,130,189), (107,174,214), (123, 142, 216), (226,103,152), (255,0,104) , (50,50,50) ]

#     # declare charts
#     chart1 = cuxfilter.charts.scatter(x='dropoff_x', y='dropoff_y', aggregate_col='ST_CASE',
#                                             color_palette=gtc_demo_red_blue_palette)
#     chart2 = cuxfilter.charts.multi_select('YEAR')
#     chart3 = cuxfilter.charts.bar('DAY_WEEK', x_label_map=label_map)
#     chart4 = cuxfilter.charts.bar('MONTH')

#     # declare dashboard
#     d = cux_df.dashboard([chart1, chart2, chart3, chart4], 
#                          layout=cuxfilter.layouts.feature_and_double_base,
#                          theme =cuxfilter.themes.rapids, #cuxfilter.themes.light, 
#                          title='Dataset')
#     charts_list = [chart1, chart2, chart3, chart4]
#     d = cux_df.dashboard(charts_list, layout_array=[
#     [1, 1, 2, 2],
#     [1, 1, 3, 4]
# ], theme=cuxfilter.themes.rapids, title="Data Layout: Custom (as left feature right double)")
# # d = cux_df.dashboard(charts_list[:4], layout=cuxfilter.layouts.left_feature_right_double, theme=cuxfilter.themes.rapids, title="Layout - left feature right double")

    
#     # run the dashboard as a webapp:
#     d._dashboard.generate_dashboard(
#         d.title, d._charts, d._theme
#     ).server_doc(doc)



In [ ]:
#launch the multiuser server
# if __name__ == '__main__':
server = Server({'/custom_dashboard': bkapp}, num_procs=1, allow_websocket_origin=["127.0.0.1:80"])
server.start()

print('running server on port '+str(port))
server.io_loop.start()

In [ ]:
#TODO: dev inverse mapping from one row of xcol_lst to one row of df_defects
#TODO: wrap ^that as foo
#TODO: recall map from one row to annihilation rates
#TODO: cp basic dashboard with two scatterplot panels
#TODO: test it works
#TODO: modify it to my desired dashboard
# - (left) umap scatter plot that let's me select a point
# - (right) plot of 

In [ ]:
#TODO: make basic plot from one value 